<a href="https://colab.research.google.com/github/keeprich/Agentic_AI/blob/main/elk_installation_amazon_linux.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/bin/bash

set -e

echo "Updating system..."
sudo dnf update -y

echo "Installing Docker..."
sudo dnf install -y docker
sudo systemctl enable docker.service
sudo systemctl start docker.service
sudo usermod -aG docker ec2-user

echo "Installing Docker Compose..."
DOCKER_COMPOSE_VERSION="2.20.2"
sudo curl -L "https://github.com/docker/compose/releases/download/v${DOCKER_COMPOSE_VERSION}/docker-compose-$(uname -s)-$(uname -m)" -o /usr/local/bin/docker-compose
sudo chmod +x /usr/local/bin/docker-compose

echo "Creating ELK stack directory..."
mkdir -p ~/elk-poc/logstash/pipeline ~/elk-poc/logs
cd ~/elk-poc

echo "Writing docker-compose.yml..."
cat > docker-compose.yml <<'EOF'
version: '3.8'

services:
  elasticsearch:
    image: docker.elastic.co/elasticsearch/elasticsearch:8.14.1
    container_name: elasticsearch
    environment:
      - discovery.type=single-node
      - ES_JAVA_OPTS=-Xms1g -Xmx1g
      - xpack.security.enabled=false
    ulimits:
      memlock:
        soft: -1
        hard: -1
    ports:
      - "9200:9200"
    volumes:
      - es_data:/usr/share/elasticsearch/data

  kibana:
    image: docker.elastic.co/kibana/kibana:8.14.1
    container_name: kibana
    ports:
      - "5601:5601"
    environment:
      - ELASTICSEARCH_HOSTS=http://elasticsearch:9200

  logstash:
    image: docker.elastic.co/logstash/logstash:8.14.1
    container_name: logstash
    volumes:
      - ./logstash/pipeline:/usr/share/logstash/pipeline
      - ./logs:/usr/share/logstash/logs:ro
    ports:
      - "5044:5044"
    depends_on:
      - elasticsearch

volumes:
  es_data:
EOF

echo "Writing Logstash pipeline config..."
cat > logstash/pipeline/logstash.conf <<'EOF'
input {
  file {
    path => "/usr/share/logstash/logs/*.log"
    start_position => "beginning"
    sincedb_path => "/dev/null"
  }
}

filter {
  grok {
    match => {
      "message" => "%{SYSLOGTIMESTAMP:timestamp} %{HOSTNAME:host} %{DATA:program}(?:\[%{POSINT:pid}\])?: %{GREEDYDATA:log_message}"
    }
  }
  date {
    match => [ "timestamp", "MMM  d HH:mm:ss", "MMM dd HH:mm:ss" ]
  }
}

output {
  elasticsearch {
    hosts => ["http://elasticsearch:9200"]
    index => "poc-logs"
  }
  stdout { codec => rubydebug }
}
EOF

echo "Creating dummy log file..."
cat > logs/dummy_syslog.log <<'EOF'
Aug 11 10:00:01 localhost CRON[1234]: (root) CMD (run-parts /etc/cron.hourly)
Aug 11 10:05:32 localhost sshd[5678]: Accepted password for user from 192.168.1.10 port 54872 ssh2
Aug 11 10:06:12 localhost sshd[5678]: Failed password for invalid user admin from 192.168.1.10 port 54873 ssh2
Aug 11 10:07:45 localhost systemd[1]: Started Session 1234 of user root.
EOF



sleep 5;


newgrp docker


sleep 5;


docker ps


sleep 5;


cd ~/elk-poc



echo "Starting ELK stack..."
docker-compose up -d

echo "Waiting for Elasticsearch to be ready..."
until curl -s http://${PUBLIC_IP}:9200 > /dev/null; do
  sleep 5
done

echo "ELK stack is up and running."

# Fetch public IP of the EC2 instance
PUBLIC_IP=$(curl -s http://169.254.169.254/latest/meta-data/public-ipv4)

echo "Access Kibana at: http://${PUBLIC_IP}:5601"
echo "Access Elasticsearch at: http://${PUBLIC_IP}:9200"
echo
echo "IMPORTANT:"
echo "- Make sure your EC2 security group allows inbound traffic on ports 5601 (Kibana) and 9200 (Elasticsearch)"
echo "- Logstash is watching logs in ~/elk-poc/logs — upload log files here to ingest"
echo "- You may need to logout/login or run 'newgrp docker' for Docker permissions to apply"
